# PPA3 Database Table Viewer
If PPA is live and people are using it, and you want to view tables, please do so through this tool rather than Arc Pro. Viewing in Pro will create a schema lock on the tables, so when a user runs the tool, it will cause an error because it cannot log the run output to the locked database tables.

This viewer gets around this issue by creating an in-memory copy of desired gdb tables.

In [1]:
import pandas as pd

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

file_gdb = r'\\arcserver-svr\D\PPA3_SVR\PPA3_GIS_SVR\PPA3_run_data.gdb'
arcpy.env.workspace = file_gdb

fc_pmaster = 'project_master'
f_uid = 'project_uid'

df_pmaster = pd.DataFrame.spatial.from_featureclass(fc_pmaster)

def esri_object_to_df(in_esri_obj, esri_obj_fields, index_field=None):
    '''converts esri gdb table, feature class, feature layer, or SHP to pandas dataframe'''
    data_rows = []
    with arcpy.da.SearchCursor(in_esri_obj, esri_obj_fields) as cur:
        for row in cur:
            out_row = list(row)
            data_rows.append(out_row)

    out_df = pd.DataFrame(data_rows, index=index_field, columns=esri_obj_fields)
    return out_df

print([t for t in arcpy.ListTables()])


['rp_artexp_vmt', 'rp_artexp_econ', 'rp_artexp_eq', 'rp_artexp_mm', 'rp_artexp_sgr', 'rp_fwy_vmt', 'rp_fwy_cong', 'rp_fwy_mm', 'rp_fwy_econ', 'rp_fwy_frgt', 'rp_fwy_saf', 'rp_artsgr_sgr', 'cd_compactdev', 'cd_mixeduse', 'cd_houschoice', 'cd_naturpres', 'rp_artexp_cong', 'rp_artexp_frgt', 'rp_artexp_saf', 'cd_trnchoice', 'cd_existgasset', 'TEST_TABLE']


## View of most recent entries into master table

In [2]:
df_pmaster.sort_values(by='time_created', ascending=False).head()

,OBJECTID,project_uid,proj_name,juris,proj_type,aadt,pci,posted_speed,len_mi,comm_type,time_created,user_email,perf_outcomes,for_review,SHAPE
273,567,ae281ca1-4f7b-4d75-b9ae-57c85c48289d,Natoma Street Drainage Phase 3 Project,City of Folsom,Arterial State of Good Repair,12500.0,65.0,35.0,0.099623,Arterials & Suburban Corridors,2023-01-25 11:45:47.000000,rneves@folsom.ca.us,Make a Safer Transportation System,NaN,"{""paths"": [[[-13489002.778, 4675303.8939], [-1..."
272,566,8593c4db-a666-4ca0-bb6c-fd833a4add94,Lindhurst and North Beale Road,Yuba County,Arterial State of Good Repair,38000.0,50.0,35.0,1.841384,Small-Town Established Communities,2023-01-25 10:26:42.000000,dpeterson@co.yuba.ca.us,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13531813.661899999, 4737904.8913..."
271,565,b04042f6-90d2-4f3f-815c-636a9f4a3ba5,Pennington Road Complete Streets Rehabilitation,City of Live Oak,Arterial State of Good Repair,9500.0,2.0,25.0,0.302245,Small-Town Established Communities,2023-01-24 15:49:09.000000,srolls@rarcivil.com,Promote Economic Prosperity; Improve Freight M...,NaN,"{""paths"": [[[-13544030.7115, 4761288.338500001..."
270,564,64ecff5c-3ced-4ce7-ae55-94c4ae3751cf,Loma Rica and South Beale Rd,Yuba County,Arterial State of Good Repair,6100.0,50.0,55.0,11.958237,Ag,2023-01-24 14:41:39.000000,dpeterson@co.yuba.ca.us,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13522128.3482, 4727493.827200003..."
269,563,6dd418e1-8a0e-4d9f-acc5-4bbe456985d7,Maintenance Paving Project,County of Sutter,Arterial State of Good Repair,2004.0,60.0,50.0,17.181492,Ag,2023-01-24 11:27:53.000001,LCarrillo@co.sutter.ca.us,Reduce Congestion; Make a Safer Transportation...,NaN,"{""paths"": [[[-13529605.142, 4716109.544], [-13..."


## Find UID values that are in any of the data tables but not in the master table

In [3]:


def get_uids(in_tbl):
    uids = []

    try:
        with arcpy.da.SearchCursor(in_tbl, field_names=[f_uid]) as cur:
            for row in cur:
                uids.append(row[0])
    except:
        print(f"{in_tbl} does not have field {f_uid}. Skipping...")
        
    return uids

data_tables = [t for t in arcpy.ListTables()]
pmaster_uids = df_pmaster[f_uid].to_list()

out_dict = {}

fake_id = 'UID_NOT_FOUND' # placeholder ID used when running tool locally for testing

for t in data_tables:
    tbl_uids = get_uids(t)
    
    uids_not_in_master = []
    for uid in tbl_uids:
        if uid not in pmaster_uids and uid != fake_id:
            uids_not_in_master.append(uid)
            
    out_dict[t] = uids_not_in_master
    if len(out_dict[t]) > 0:
        print(f"table {t} has the follwing UIDs not in the project_master table: {out_dict[t]}")
        
total_missing = sum([len(v) for tbl, v in out_dict.items()])
if total_missing == 0: print("no IDs found in subreport tables that are not also in project_master")
    

TEST_TABLE does not have field project_uid. Skipping...
no IDs found in subreport tables that are not also in project_master


## Show specific subreport data and for specific projects

In [4]:

subrpt_tbl = 'rp_fwy_econ'
project_uid = None

fields = [f.name for f in arcpy.ListFields(subrpt_tbl)]

#uncomment if you want to filter which fields appear
# fields = [f_uid, 'crash_cnt', 'crash_100mvmt'] # [f.name for f in arcpy.ListFields(subrpt_tbl)]

df_master_fields = [f_uid, 'time_created', 'user_email', 'len_mi', 'aadt']
df_subrpt = esri_object_to_df(subrpt_tbl, esri_obj_fields=fields, index_field=None)


df_jn = df_pmaster[df_master_fields].merge(df_subrpt, on=f_uid, suffixes=('','_y')) \
    .sort_values(by='time_created', ascending=False)
df_jn.head()

,project_uid,time_created,user_email,len_mi,aadt,OBJECTID,acc_drive_alljob,acc_drive_edu
57,8e0c24f7-56ff-4a6e-9c40-a1cdbef49e6a,2023-01-23 10:46:17,Justin.lau@Cityofwoodland.org,4.687798,7077.0,101,222922.640625,180.939560
56,ae44b9cf-8bcb-4316-80b6-76cc36fba74b,2023-01-20 15:18:02,gballard-rosa@sacog.org,3.051947,163000.0,100,713744.312500,729.444458
55,dc6f1f35-648b-46f4-aed0-f438af0d99f0,2023-01-20 15:13:15,gballard-rosa@sacog.org,2.497230,117000.0,99,708543.562500,731.922913
54,dbce3f3d-ba7f-4cab-b323-dd4c2a33f7b4,2023-01-20 15:12:13,gballard-rosa@sacog.org,9.947685,157000.0,98,554717.187500,518.484619
53,44e1b622-0202-431a-8cf7-2dd018e6e00a,2023-01-20 15:10:57,gballard-rosa@sacog.org,15.544632,150400.0,97,607230.562500,589.432068


In [42]:
df_jn.loc[df_jn.crash_100mvmt > 100].head()

,project_uid,time_created,user_email,len_mi,aadt,crash_cnt,crash_100mvmt
21,55160fbd-9617-4c09-a05e-da8819a2b71c,2022-12-06 13:47:37,jhong@sacog.org,3.670994,5.0,136,463089.843750
45,fbc41452-2ed7-44ce-b1d1-3563bad0f73c,2023-01-17 06:06:46,rbissegger@markthomas.com,1.861210,58600.0,36,130.566345
